<img src="utils/mediapipelogo.png" width="700" height="100">

# Biceps Reps Tracker using Media Pipe

### Steps to be followed
- Install dependencies
- Make Detections
- Determining Joints
- Calculating Angles
- Curl Counter Custom function

In [5]:
import mediapipe as mp
import cv2
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose # grabbing the pose estimation model from MediaPipe 

## Raw Feed from camera ( just for testing webcam is working or not )

In [2]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = cv2.resize(frame, (1200, 790))
#     cv2.rectangle(frame, (0,0), (1200, 790), (0, 0, 0), -1) # just for creating black screen.
    cv2.imshow('MediaPipe Feeding', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [3]:
cap.release()
cv2.destroyAllWindows()

# Making Detections

In [6]:
cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose :
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # just for optimization purpose
        image.flags.writeable = False
        
        # detections storing into results
        results = pose.process(image)
        # print(results) # Printing the results variable (optional) for analysis
        
        # just for optimization purpose
        image.flags.writeable = True
        
        # recolouring back to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # rendering the detections using .draw_landmarks and DrawingSpec
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        
        
        
        cv2.imshow('MediaPipe feeding', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
            

cap.release()
cv2.destroyAllWindows()

In [8]:
mp_drawing.draw_landmarks??

# Determining the Joints


<img src ="utils/bodypose.png" stype="height:350px">

###  **There are a total of 33 landmarks in total starting from the index 0 . Each of these represent the different joints within the pose**

In [9]:
cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose :
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # detections storing into results
        results = pose.process(image)
        #print(results)
        
        # recolouring back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        # Extract Landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            # print(landmarks) # printing the landmarks from the result object
        except:
            pass
        
        # render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        
        
        
        cv2.imshow('MediaPipe feeding', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
            

cap.release()
cv2.destroyAllWindows()

In [10]:
len(landmarks)

33

In [11]:
# mp_pose.PoseLandmark will give the whole landmarks that media pipe detects
for landmark in mp_pose.PoseLandmark:
    print(landmark) 

PoseLandmark.NOSE
PoseLandmark.LEFT_EYE_INNER
PoseLandmark.LEFT_EYE
PoseLandmark.LEFT_EYE_OUTER
PoseLandmark.RIGHT_EYE_INNER
PoseLandmark.RIGHT_EYE
PoseLandmark.RIGHT_EYE_OUTER
PoseLandmark.LEFT_EAR
PoseLandmark.RIGHT_EAR
PoseLandmark.MOUTH_LEFT
PoseLandmark.MOUTH_RIGHT
PoseLandmark.LEFT_SHOULDER
PoseLandmark.RIGHT_SHOULDER
PoseLandmark.LEFT_ELBOW
PoseLandmark.RIGHT_ELBOW
PoseLandmark.LEFT_WRIST
PoseLandmark.RIGHT_WRIST
PoseLandmark.LEFT_PINKY
PoseLandmark.RIGHT_PINKY
PoseLandmark.LEFT_INDEX
PoseLandmark.RIGHT_INDEX
PoseLandmark.LEFT_THUMB
PoseLandmark.RIGHT_THUMB
PoseLandmark.LEFT_HIP
PoseLandmark.RIGHT_HIP
PoseLandmark.LEFT_KNEE
PoseLandmark.RIGHT_KNEE
PoseLandmark.LEFT_ANKLE
PoseLandmark.RIGHT_ANKLE
PoseLandmark.LEFT_HEEL
PoseLandmark.RIGHT_HEEL
PoseLandmark.LEFT_FOOT_INDEX
PoseLandmark.RIGHT_FOOT_INDEX


In [12]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value] # indexing and extracting the coordinates of particular body part

x: 0.7497252
y: 0.84485066
z: -0.118524075
visibility: 0.9999505

In [13]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value] # indexing and extracting the coordinates

x: 0.81655914
y: 1.136943
z: 0.03518168
visibility: 0.18905202

In [14]:
landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value] # indexing and extracting the coordinates

x: 0.8252918
y: 1.3855854
z: -0.39320654
visibility: 0.030742262

In [15]:
mp_pose.PoseLandmark.RIGHT_SHOULDER.value # index of the landmarks

12

# Calculating Angles

- Custom Function to calculate angle between any 3 points in the detected keypoints

In [11]:

def calculate_angles(a, b, c):
    a = np.array(a) # First a shoulder
    b = np.array(b) # End b elbow
    c = np.array(c) # Mid c wrist
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360-angle
        
    return angle
    

In [12]:
shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y
wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y

In [13]:
print(shoulder)
print(elbow)
print(wrist)

(0.6845836043357849, 0.7624749541282654)
(0.7437582015991211, 1.0672228336334229)
(0.782760500907898, 1.3047481775283813)


In [14]:
calculate_angles(shoulder, elbow, wrist) # executing the function for testing purpose

178.33621069148893

## Calculating the angle between shoulder, elbow, wrist

In [15]:
cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose :
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # detections storing into results
        results = pose.process(image)
        #print(results)
        
        # recolouring back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        # Extract Landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            # print(landmarks)
            # getting the coordinates
            shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
            elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y
            wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
            
            # calculating the angle
            angle = calculate_angles(shoulder, elbow, wrist)
            
            
            # Visualizing the angle
            cv2.putText(image, str(angle),
                       tuple(np.multiply(elbow, [640,480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
                            )
        except:
            pass
        
        # render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        
        
        
        cv2.imshow('MediaPipe feeding', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
            

cap.release()
cv2.destroyAllWindows()

# Custom function Curl Counter 

- Custom function to count the no. of repetitions

In [16]:
cap = cv2.VideoCapture(0)

# variables for counting exercises
counter = 0
stage = None

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose :
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # detections storing into results
        results = pose.process(image)
        #print(results)
        
        # recolouring back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        # Extract Landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            # print(landmarks)
            # getting the coordinates
            shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
            elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y
            wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
            
            # calculating the angle
            angle = calculate_angles(shoulder, elbow, wrist)
            
            
            # Visualizing the angle
            cv2.putText(image, str(angle),
                       tuple(np.multiply(elbow, [640,480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
                            )
            # Curl Counter Logic
            if angle > 160:
                stage = 'down'
            if angle < 30 and stage == 'down':
                stage = 'up'
                counter +=1
                print(counter)
            
            
            
        except:
            pass
        
        
        cv2.rectangle(image, (0,0), (300,73), (11, 13, 13), -1)
        
        # Counter data
        cv2.putText(image, 'REPS : ', (15, 12),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        # stage data
        cv2.putText(image, 'STAGE : ', (110, 12),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(image, str(stage), (90, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        
        # render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        
        
        
        cv2.imshow('MediaPipe feeding', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
            

cap.release()
cv2.destroyAllWindows()

1
2
3
